In [ ]:
import pydantic
import sys
from pydantic import BaseModel
from datasets import load_dataset
import asyncio
from synthetic_data.generation import RemoteModel, get_generation_wrapper


sys.path.append("..")
%load_ext autoreload
%autoreload 2
from trl_wrapper.trainer_wrapper import TrainerWrapper, PLAYWRIGHT_CONFIG, SMOL_LM_135M

dataset = load_dataset("PygmalionAI/PIPPA", "pippa_deduped", trust_remote_code=True)[
    "train"
]
dataset_pl = dataset.to_polars()


In [ ]:
import json
from typing import Sequence
from openai.types.chat.chat_completion_message_param import ChatCompletionMessageParam
from synthetic_data.screenplay_parser import Scene


def format_conversation_oai(sample: dict) -> Sequence[ChatCompletionMessageParam]:
    # TODO handle interpolation and character intros
    conversation, bot_name = sample["conversation"], sample["bot_name"]
    out_conv: Sequence[dict] = []
    for msg, is_human in zip(conversation["message"], conversation["is_human"]):
        out_conv.append({"role": bot_name if not is_human else "User", "content": msg})

    return [
        {
            "role": "system",
            "content": "Extract the dialogue, actions, and descriptions from the conversation given by the user.",
        },
        {"role": "user", "content": json.dumps(out_conv)},
    ]


In [ ]:
from synthetic_data.generation import GenWrapperArgs
from typing import List
from enum import Enum



class SceneElementType(Enum):
    SCENE_HEADING = "scene_heading"
    ACTION = "action"
    DIALOGUE = "dialogue"
    TRANSITION = "transition"


class SceneElement(BaseModel):
    type: SceneElementType
    content: str
    character: str | None = None

class Output(BaseModel):
    items: List[SceneElement]

model: str = RemoteModel.GPT_4O_MINI.value
generation_wrapper = get_generation_wrapper(model, GenWrapperArgs(response_format=Output)) # type: ignore

conv = format_conversation_oai(dataset[5])

completions = await generation_wrapper.generate([conv])
print(completions)

In [ ]:
json.loads(completions[0])